In [145]:
# !pip install xlrd

%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
df = pd.read_csv('Data/Clean/project_data_cleaned.csv', encoding='latin1')
df = df.drop("Unnamed: 0", axis = 1)

df.head().T

C:\Users\mattk\anaconda3\envs\msca\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
case_status,Denied,Certified-Expired,Denied,Certified,Denied
received_date,9/3/2009,3/29/2010,8/10/2010,2/3/2011,3/10/2011
decision_date,4/30/2020,10/18/2019,4/30/2020,5/28/2020,6/17/2020
employer_city,CHICAGO,GERMANTOWN,N. VALLEY STREAM,CAMPBELL,YONKERS
employer_state_province,ILLINOIS,MARYLAND,NEW YORK,CALIFORNIA,NEW YORK
employer_num_employees,2,0,3,29,43
emp_contact_city,chicago,GERMANTOWN,N. VALLEY STREAM,Sunnyvale,YONKERS
emp_contact_state_province,ILLINOIS,MARYLAND,NEW YORK,CALIFORNIA,NEW YORK
emp_contact_postal_code,60647,20874,11580,94086,10703
agent_attorney_name,Christopher E Kurczaba,WINSTON W TSAI,DANIEL J SULLIVAN,Sonal Kinger,VERNON L DUTTON


In [147]:
df.shape

(71942, 33)

In [151]:
df['received_date'] = pd.to_datetime(df['received_date'], format = '%m/%d/%Y')
df['decision_date'] = pd.to_datetime(df['decision_date'], format = '%m/%d/%Y')

df['wait_time'] = (df['decision_date'] - df['received_date'])/np.timedelta64(1, "M")

df['decision_month'] = df['decision_date'].dt.to_period('M')

df['age_of_business'] = 2020 - df.employer_year_commenced_business
df['naics_group'] = df.naics_code.astype(str).str[:2]

df['pw_wage'] = df.pw_wage.str.replace(',', '')
df['pw_wage'] = pd.to_numeric(df.pw_wage)
df['pw_wage_adj'] = np.where(df.pw_unit_of_pay == "Hour", df.pw_wage*2080, df.pw_wage)
df['pw_wage_adj'] = np.where(df.pw_unit_of_pay == "Week", df.pw_wage*52, df.pw_wage_adj)
df['pw_wage_adj'] = np.where(df.pw_unit_of_pay == "Month", df.pw_wage*12, df.pw_wage_adj)
df['pw_wage_adj'] = np.where(df.pw_unit_of_pay == "Bi-Weekly", df.pw_wage*26, df.pw_wage_adj)

- 11	Agriculture, Forestry, Fishing and Hunting
- 21	Mining
- 22	Utilities
- 23	Construction
- 31	Manufacturing
- 32  Manufacturing
- 33  Manufacturing
- 42	Wholesale Trade
- 44  Retail Trade
- 45	Retail Trade
- 48  Transportation and Warehousing
- 49	Transportation and Warehousing
- 51	Information
- 52	Finance and Insurance
- 53	Real Estate Rental and Leasing
- 54	Professional, Scientific, and Technical Services
- 55	Management of Companies and Enterprises
- 56	Administrative and Support and Waste Management and Remediation Services
- 61	Educational Services
- 62	Health Care and Social Assistance
- 71	Arts, Entertainment, and Recreation
- 72	Accommodation and Food Services
- 81	Other Services (except Public Administration)
- 92	Public Administration

In [156]:
pd.crosstab(df.decision_month, df.case_status)

case_status,Certified,Certified-Expired,Denied,Withdrawn
decision_month,,,,
2019-10,0,7029,408,290
2019-11,0,4980,236,274
2019-12,0,4063,326,327
2020-01,4453,255,235,307
2020-02,3369,0,348,238
2020-03,7028,0,259,299
2020-04,13130,0,299,287
2020-05,11554,0,332,463
2020-06,10374,0,439,340
